Denote each group as a pair $(s,\, w)$ where $s$ is the total number of balls and $w$ is the number of white balls. Define a total order on all such pairs using the usual tuple ordering rule: $(s,\,w) \ge (s',\,w')$ iff $s > s'$ or $s = s'$ and $w \ge w'$.

Now, let $f(s, w, s_0, w_0)$ be the total number of groupings of $s$ balls with $w$ being white, with the constraint that the largest group is no greater than $(s_0, w_0)$ by the total order defined above. Then

$$f(s,\, w,\, s_0,\, w_0) = \sum_{(s_1,\,w_1):\ s_1 \le s,\, w_1 \le w,\, (s_1,\,w_1) \le (s_0,\,w_0)} f(s-s_1,\, w-w_1,\, s_1,\, w_1).$$

The base case ares

$$f(0, 0, s_0, w_0) = 1$$

and

$$
f(s, w, 1, 0) = 
\begin{cases}
1, & w = 0,\\
0, & w > 0.
\end{cases}
$$

We can massage $f(s-s_1,\, w-w_1,\, s_1,\, w_1)$ a bit so that in of our recursive calls we have $s \ge w \ge w_0$ and $s \ge s_0 \ge w_0$ (this way the recursive calls are more caching-friendly).

The Python implementation for this is kind of slow, ~30s. One obvious optimization would be turning recursive calls into flat nested loops.

In [1]:
#!/usr/bin/env python3

import functools


@functools.lru_cache(maxsize=None)
def f(s, w, s0, w0):
    assert s >= w >= w0 and s >= s0 >= w0
    if (s0, w0) == (1, 0):
        return 1 if w == 0 else 0
    count = 0
    for s1 in range(1, min(s, s0) + 1):
        if s1 == s:
            # All in one group.
            if w0 >= w:
                count += 1
            continue
        if s1 == s0:
            max_w1 = min(w, w0)
        else:
            max_w1 = min(w, s1)
        for w1 in range(max_w1 + 1):
            rs = s - s1
            rw = w - w1
            if rs < rw:
                continue
            new_s0, new_w0 = min((rs, rw), (s1, w1))
            new_w0 = min(new_w0, rw)
            count += f(rs, rw, new_s0, new_w0)
    return count


def main():
    b = 60
    w = 40
    print(f(b + w, w, b + w, w))


if __name__ == "__main__":
    main()


83735848679360680
